In [2]:
import numpy as np
import math

In [161]:
rng = np.random.default_rng()

In [213]:
class AntColony:
    def __init__(self, number_of_ants = None, alpha = 1, beta = 1,
                 starting_pheromone = 1, Q = 1):
        if number_of_ants is not None and number_of_ants <= 0:
            raise Exception("Negative number of ants")
        if number_of_ants is not None and number_of_ants != math.floor(number_of_ants):
            raise Exception("Number of ants is not an intefer")
        if alpha < 0:
            raise Exception("Alpha parameter has to be positive")
        if beta < 0:
            raise Exception("Beta parameter has to be positive")
        if starting_pheromone < 0:
            raise Exception("starting_pheromone parameter has to be positive")
        if Q < 0:
            raise Exception("Q parameter has to be positive")

        self.number_of_ants = number_of_ants
        self.alpha = alpha
        self.beta = beta
        self.starting_pheromone = starting_pheromone
        self.Q = Q
    
    def set_problem(self, coordinates, request, capasity, s_max):
        if coordinates.shape[1] != 2:
            raise Exception("coordinates are not in IR^2")
        if coordinates.shape[0]-1 != request.shape[0]:
            raise Exception("Different number of coordinates and requests")
        if np.any(request <= 0):
            raise Exception("There is non positive request")
        if np.any(request > capasity):
            raise Exception("There is request bigger than capasity. This problem is unsolvable!")
        
        self.problem_size = coordinates.shape[0]
        self.coordinates = coordinates
        self.request = request
        self.capasity = capasity
        
        if self.number_of_ants is None:
            self.number_of_ants = 2 * self.problem_size
        
        self.pheromone_restart()
        self.calculate_distance_matrix()
        self.calculate_transition_matrix()
        
        if np.any(self.distance_matrix * 2 > s_max):
            raise Exception("There is distance bigger than max_s / 2. This problem is unsolvable! Check antColony.distance_matrix")
        
    def calculate_distance_matrix(self):
        self.distance_matrix = np.zeros((self.problem_size, self.problem_size))
        
        for i in range(0, self.problem_size):
            for j in range(i+1, self.problem_size): # only above the diagonal
                d = np.linalg.norm( self.coordinates[i,:] - self.coordinates[j,:] )
                self.distance_matrix[i, j] = d
                self.distance_matrix[j, i] = d
    
    def calculate_transition_matrix(self):
        # non standardize Probability
        modified_distance_matrix = self.distance_matrix + np.eye(self.problem_size) # diagonal will later be set to 0
        self.T_P = self.pheromone_matrix**self.alpha*(1/modified_distance_matrix)**self.beta
        np.fill_diagonal(self.T_P, 0)
    
    def pheromone_restart(self):
        self.pheromone_matrix = self.starting_pheromone * (np.ones((self.problem_size,
                                                                    self.problem_size)) - np.eye(self.problem_size))
        
    def ant_find_path(self): # return (list of nodes, number of times to start from warehouse)
        pass
        #return [0, 4, 2, 0, 1, 3, 0], 2
    
    def ant_find_circle(self, visited_nodes):
        pass
    
    def calculate_cost(self, path):
        pass
        #return rng.uniform()
    
    def pheromone_modify(self, paths):
        pass # TODO, trzeba utrzymac symetrię macierzy
    
    def single_iteration(self):
        paths = [None for i in range(self.number_of_ants)]
        num_of_cycles = [None for i in range(self.number_of_ants)]
        for i in range(self.number_of_ants):
            paths[i], num_of_cycles[i] = self.ant_find_path()
        costs = np.array([self.calculate_cost(paths[i]) for i in range(self.number_of_ants)])
        
        i_min_cost = np.argmin(costs)
        if costs[i_min_cost] < self.best_cost:
            self.best_path = paths[i_min_cost]
            self.best_cost = costs[i_min_cost]
            self.best_cycles = num_of_cycles[i_min_cost]
            print("New best solution: cost = {:.2f} and uses {} trucks".format(self.best_cost, self.best_cycles))
        
        self.pheromone_modify(paths)
    
    def optimize(self, max_iter):
        self.best_cost = np.inf
        for i in range(max_iter):
            self.single_iteration()

In [214]:
antColony = AntColony(number_of_ants=None, alpha=1, beta=1, starting_pheromone = 1, Q = 1)

In [215]:
request = np.array([10, 10, 10, 10, 10])

In [216]:
coordinates = np.array([[365, 689], [365.4, 689.34], [365.3, 689.5], [365.3, 689.3], [365.2, 689.2], [365.1, 689.1]])

In [217]:
antColony.set_problem(coordinates, request, 100, 1.2)

In [218]:
antColony.T_P

array([[0.        , 1.90484829, 1.71498585, 2.3570226 , 3.53553391,
        7.07106781],
       [1.90484829, 0.        , 5.2999894 , 9.28476691, 4.0961596 ,
        2.60289603],
       [1.71498585, 5.2999894 , 0.        , 5.        , 3.16227766,
        2.23606798],
       [2.3570226 , 9.28476691, 5.        , 0.        , 7.07106781,
        3.53553391],
       [3.53553391, 4.0961596 , 3.16227766, 7.07106781, 0.        ,
        7.07106781],
       [7.07106781, 2.60289603, 2.23606798, 3.53553391, 7.07106781,
        0.        ]])

In [219]:
antColony.number_of_ants

12

In [227]:
antColony.optimize(30)

New best solution: cost = 0.10 and uses 2 trucks
New best solution: cost = 0.03 and uses 2 trucks
New best solution: cost = 0.02 and uses 2 trucks
New best solution: cost = 0.01 and uses 2 trucks
New best solution: cost = 0.00 and uses 2 trucks
New best solution: cost = 0.00 and uses 2 trucks
